In [1]:
import torchvision.models as models

model = models.resnet18(pretrained=False)
print(model.conv1)
print(model.maxpool)


Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)


/Users/benjipro/Repositories/mli-digit-recogniser/.venv/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/benjipro/Repositories/mli-digit-recogniser/.venv/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [6]:
import sys
from pathlib import Path
from src.modeling.predict import get_resnet18_mnist
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch

# Add parent directory to sys.path for imports
sys.path.append(str(Path.cwd().parent))

# Set device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Build correct model path relative to notebook directory
model_path = Path.cwd().parent / "src" / "models" / "resnet18_mnist.pth"

# Load model and weights
model = get_resnet18_mnist().to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# MNIST preprocessing
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load MNIST test dataset
test_dataset = datasets.MNIST(root="data", train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Evaluate accuracy
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


100%|██████████| 9.91M/9.91M [00:00<00:00, 10.9MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 278kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 3.21MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.38MB/s]


Test Accuracy: 99.19%


- see above

In [ ]:
sadsa